# Douyin Live Recorder

Notebook นี้ใช้สำหรับรัน DouyinLiveRecorder เพื่อบันทึกวิดีโอไลฟ์จาก Douyin (TikTok จีน)

In [ ]:
# Clone repositories
!git clone https://github.com/ihmily/DouyinLiveRecorder.git
!git clone https://github.com/chaiz64/l777k.git

In [ ]:
# Copy configuration files
!cp l777k/config.ini DouyinLiveRecorder/config/
!cp l777k/URL_config.ini DouyinLiveRecorder/config/

In [ ]:
# เข้าไปยังโฟลเดอร์โปรเจค
%cd DouyinLiveRecorder

# ติดตั้ง dependencies
!pip3 install -r requirements.txt

In [ ]:
# ติดตั้ง ffmpeg
!apt update
!apt install ffmpeg -y

In [ ]:
# รันโปรแกรม
!python main.py

In [ ]:
import shutil
from google.colab import files

# กำหนดชื่อไฟล์ ZIP
zip_filename = "/content/douyin_live_backup.zip"
source_folder = "/content/DouyinLiveRecorder/downloads/抖音直播"

# บีบอัดโฟลเดอร์เป็นไฟล์ ZIP
shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', source_folder)

# ดาวน์โหลดไฟล์ ZIP
files.download(zip_filename)


In [ ]:
import os
import IPython.display as display

# ฟังก์ชันแปลงขนาดไฟล์เป็น GB หรือ TB
def convert_size(size_in_bytes):
    if size_in_bytes < 1024:
        return f"{size_in_bytes} B"
    elif size_in_bytes < 1024**2:
        return f"{size_in_bytes / 1024:.2f} KB"
    elif size_in_bytes < 1024**3:
        return f"{size_in_bytes / 1024**2:.2f} MB"
    elif size_in_bytes < 1024**4:
        return f"{size_in_bytes / 1024**3:.2f} GB"
    else:
        return f"{size_in_bytes / 1024**4:.2f} TB"

# กำหนดโฟลเดอร์หลักที่เก็บไฟล์
base_folder = "/content/DouyinLiveRecorder/downloads/抖音直播"

# ตรวจสอบว่าโฟลเดอร์มีอยู่หรือไม่
if not os.path.exists(base_folder):
    print("❌ ไม่พบโฟลเดอร์ที่กำหนด")
else:
    # ค้นหาไฟล์ทั้งหมดในโฟลเดอร์และโฟลเดอร์ย่อย
    file_paths = []
    for root, _, files in os.walk(base_folder):
        for file in files:
            if file.endswith(('.mp4', '.ts')):  # เลือกเฉพาะไฟล์ .mp4 และ .ts
                full_path = os.path.join(root, file)
                relative_path = full_path.replace(base_folder + "/", "")  # แสดงพาธย่อ
                file_paths.append((full_path, relative_path))

    # ตรวจสอบว่าพบไฟล์หรือไม่
    if not file_paths:
        print("❌ ไม่พบไฟล์ .mp4 หรือ .ts ในโฟลเดอร์")
    else:
        print(f"✅ พบ {len(file_paths)} ไฟล์ในโฟลเดอร์ '{base_folder}'\n")

        # แสดงลิงก์ดาวน์โหลดและขนาดไฟล์
        for full_path, relative_path in sorted(file_paths):
            file_size = os.path.getsize(full_path)
            file_size_str = convert_size(file_size)  # แปลงขนาดไฟล์เป็นหน่วยที่เหมาะสม
            display.display(display.HTML(
                f'<a href="/files{full_path}" download="{relative_path}" target="_blank">📥 ดาวน์โหลด {relative_path} ({file_size_str})</a><br>'
            ))


In [ ]:
import os
from google.colab import files
import time

def scan_and_download_files(config):
    """
    Scans a specified directory and its subdirectories for files,
    then downloads them one by one to the local machine, up to a specified limit.

    Args:
        config (dict): A dictionary containing configuration parameters,
                       including 'base_download_directory' and 'DOWNLOAD_LIMIT'.
    """
    base_directory = config.get('base_download_directory')
    download_limit = config.get('DOWNLOAD_LIMIT', 2) # Default to 2 if not specified

    if not base_directory:
        print("Error: 'base_download_directory' is not specified in the configuration.")
        return

    if not os.path.isdir(base_directory):
        print(f"Error: Directory '{base_directory}' not found.")
        print("Please ensure the path is correct and the directory exists in your Colab environment.")
        return

    print(f"Scanning for files in: {base_directory}")
    found_files = []

    # Use os.walk to traverse directories and subdirectories
    for root, _, filenames in os.walk(base_directory):
        for filename in filenames:
            full_path = os.path.join(root, filename)
            found_files.append(full_path)

    if not found_files:
        print(f"No files found in '{base_directory}' or its subdirectories.")
        return

    print(f"\nFound {len(found_files)} file(s) in total:")
    for i, f in enumerate(found_files):
        # Displaying only the relative path for readability
        relative_path = os.path.relpath(f, base_directory)
        print(f"  {i+1}. {relative_path}")

    # --- Apply download limit ---
    files_to_download = found_files[:download_limit]
    remaining_files_count = len(found_files) - len(files_to_download)

    if not files_to_download:
        print("No files selected for download based on the limit. Adjust 'DOWNLOAD_LIMIT' if needed.")
        return

    print(f"\nSelected {len(files_to_download)} file(s) for download (max {download_limit}):")
    for i, f in enumerate(files_to_download):
        relative_path = os.path.relpath(f, base_directory)
        print(f"  - {relative_path}")

    # Optional: Ask for confirmation before downloading
    if len(files_to_download) > 0:
        user_input = input("\nDo you want to proceed with downloading the selected files? (yes/no): ").lower().strip()
        if user_input != 'yes':
            print("Download cancelled by user.")
            return

    print("\nStarting download process...")
    for i, file_path in enumerate(files_to_download):
        try:
            relative_path = os.path.relpath(file_path, base_directory)
            print(f"[{i+1}/{len(files_to_download)}] Downloading: {relative_path}")

            # The files.download() function handles the actual download to your local machine
            files.download(file_path)
            print(f"  --> Download initiated for '{relative_path}'.")

            # Add a small delay between downloads to prevent issues with browser download limits
            time.sleep(2) # Adjust delay as needed (e.g., 1-5 seconds)

        except FileNotFoundError:
            print(f"  Error: File not found at '{file_path}'. Skipping.")
        except Exception as e:
            print(f"  An unexpected error occurred during download of '{file_path}': {e}. Skipping.")

    print("\nDownload process completed.")
    print("Please check your browser's download folder for the downloaded files.")

    if remaining_files_count > 0:
        print(f"\n{remaining_files_count} file(s) were skipped due to the download limit.")
        print("You can adjust the 'DOWNLOAD_LIMIT' in the configuration to download more files in subsequent runs.")

# --- Configuration ---
# Define the base directory you want to scan and download from
# Based on your previous query, this would be:
config = {
    'base_download_directory': '/content/DouyinLiveRecorder/downloads/抖音直播/',
    'DOWNLOAD_LIMIT': 2 # Maximum number of files to download in one run
}

# --- Run the function ---
scan_and_download_files(config)